In [2]:
import mysql.connector
import pandas as pd
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()
db_password = os.getenv('password')

# --- 1. MySQL 연결 및 데이터 추출 함수 ---
# (이 부분은 이전 코드와 동일합니다.)
def get_db_connection():
    """DB 연결 객체를 반환합니다."""
    try:
        conn = mysql.connector.connect(
            host="127.0.0.1",
            user="root",
            password=db_password,
            database="new_schema"
        )
        return conn
    except mysql.connector.Error as err:
        print(f"Database Connection Error: {err}")
        return None

def load_data_to_df(conn, table_name, query="SELECT * FROM {table_name}"):
    """테이블 이름과 쿼리를 받아 Pandas DataFrame을 반환합니다."""
    
    final_query = query.format(table_name=table_name)
    
    try:
        # SQL 쿼리를 DataFrame으로 바로 읽습니다. (전체 데이터를 로드할 경우 시간이 소요될 수 있습니다.)
        df = pd.read_sql(final_query, conn)
        print(f"✅ {table_name} 데이터 로드 성공 (총 {len(df)}개 행)")
        return df
    except Exception as e:
        print(f"❌ {table_name} 데이터 로드 실패: {e}")
        return pd.DataFrame() 

# --- 2. 데이터 정리 및 통합 메인 로직 ---
def prepare_ui_data():
    conn = get_db_connection()
    if conn is None:
        return {} 

    data_frames = {}
    
    # a) 버스 노선 데이터 (지도 활용)
    data_frames['bus'] = load_data_to_df(conn, 'bus_routes2', 
                                          query="SELECT 정류장명, 경도, 위도 FROM {table_name}")
    
    # b) 사업체 위치 데이터 (지도 활용)
    data_frames['business_coords'] = load_data_to_df(conn, 'chuncheon_businesses_with_coords',
                                          query="SELECT 업종, 업소명, 위도, 경도 FROM {table_name}")
    
    # c) 상권 기본 정보 (대시보드 KPI 및 JOIN 기준)
    data_frames['area_info'] = load_data_to_df(conn, 'commercial_area',
                                               query="""SELECT 상권업종대분류명, 시군구명, 경도, 위도 FROM {table_name}""")
    
    # d) 상권별 사업체 상세 정보 (JOIN 대상)
    data_frames['sanggwon_biz'] = load_data_to_df(conn, 'sanggwon_businesses',
                                               query="SELECT 상호명, 업종명, 주소 FROM {table_name}")
    
    # ⭐️ e) 강원도 사업체 상세 목록 (비교 분석 및 목록 표시용)
    data_frames['kangwon_biz'] = load_data_to_df(conn, 'kangwondo_businesses',
                                               query="SELECT 업종, 업소명, 주소, 연락처 FROM {table_name}")
    
    # --- 데이터 정리 및 컬럼명 통일 ---
    
    # 1. 버스 데이터 컬럼 통일
    if not data_frames['bus'].empty:
        data_frames['bus'].columns = ['station_name', 'lon', 'lat']
        data_frames['bus'] = data_frames['bus'].dropna(subset=['lon', 'lat'])

    # 2. 사업체 위치 컬럼 통일
    if not data_frames['business_coords'].empty:
        data_frames['business_coords'].columns = ['type', 'name', 'lat', 'lon']
        data_frames['business_coords'] = data_frames['business_coords'].dropna(subset=['lat', 'lon'])
        
    # 3. 강원도 사업체 컬럼 통일 (지도 데이터와 구조가 다르므로 분리 유지)
    if not data_frames['kangwon_biz'].empty:
        data_frames['kangwon_biz'].columns = ['type', 'name', 'address', 'contact']
        
    # (나머지 area_info 및 sanggwon_biz 정리 로직 생략, 기존과 동일)
        
    conn.close()
    
    print("\n--- 데이터 정리 완료 ---")
    
    # 최종적으로 UI에 전달할 핵심 데이터셋 딕셔너리 반환
    return {
        "bus_stations": data_frames['bus'].to_dict('records'),
        "businesses": data_frames['business_coords'].to_dict('records'),
        "all_businesses_list": data_frames['kangwon_biz'].to_dict('records'), # 새로운 데이터셋
        "area_info": data_frames['area_info'].to_dict('records'),
        "sanggwon_details": data_frames['sanggwon_biz'].to_dict('records')
    }

# --- 실행 ---
if __name__ == "__main__":
    prepared_data = prepare_ui_data()
    
    print(f"\n준비된 최종 데이터셋 키: {list(prepared_data.keys())}")
    print("\n--- 강원도 사업체 목록 데이터 샘플 (활용 예시) ---")
    print(prepared_data['all_businesses_list'][:2])
    print("\n이제 이 데이터를 활용하여 UI를 구축할 수 있습니다.")

/var/folders/n7/7zp7j9413xsgfgqknxqw7h880000gn/T/ipykernel_11751/1591753474.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(final_query, conn)


✅ bus_routes2 데이터 로드 성공 (총 31560개 행)
✅ chuncheon_businesses_with_coords 데이터 로드 성공 (총 34개 행)
✅ commercial_area 데이터 로드 성공 (총 17497개 행)
✅ sanggwon_businesses 데이터 로드 성공 (총 107631개 행)
✅ kangwondo_businesses 데이터 로드 성공 (총 356개 행)

--- 데이터 정리 완료 ---

준비된 최종 데이터셋 키: ['bus_stations', 'businesses', 'all_businesses_list', 'area_info', 'sanggwon_details']

--- 강원도 사업체 목록 데이터 샘플 (활용 예시) ---
[{'type': '한식', 'name': '청재', 'address': '강원도 춘천시 서면 박사로 927', 'contact': '033-255-2867'}, {'type': '한식', 'name': '두리감자탕', 'address': '강원도 춘천시 후평동 748-4', 'contact': '033-244-2847'}]

이제 이 데이터를 활용하여 UI를 구축할 수 있습니다.


In [6]:
import streamlit as st
import pandas as pd
import mysql.connector
import os
import folium
from dotenv import load_dotenv
from streamlit_folium import folium_static

# --- 0. 환경 설정 및 DB 연결 함수 ---
load_dotenv()
db_password = os.getenv('password')

@st.cache_resource # DB 연결은 리소스로 캐싱
def get_db_connection():
    """DB 연결 객체를 반환합니다."""
    try:
        conn = mysql.connector.connect(
            host="127.0.0.1",
            user="root",
            password=db_password,
            database="new_schema"
        )
        return conn
    except mysql.connector.Error as err:
        st.error(f"데이터베이스 연결 오류: {err}")
        st.stop()
        return None

def load_data_to_df(conn, table_name, query="SELECT * FROM {table_name}"):
    """테이블 이름과 쿼리를 받아 Pandas DataFrame을 반환합니다."""
    try:
        final_query = query.format(table_name=table_name)
        df = pd.read_sql(final_query, conn)
        return df
    except Exception as e:
        st.warning(f"테이블 '{table_name}' 로드 실패: {e}")
        return pd.DataFrame()

# --- 1. 데이터 통합 및 정리 함수 (캐싱 적용) ---
@st.cache_data
def prepare_ui_data():
    conn = get_db_connection()
    
    # a) 사업체 위치 데이터 (지도 활용)
    df_biz_coords = load_data_to_df(conn, 'chuncheon_businesses_with_coords',
                                    query="SELECT 업종, 업소명, 위도, 경도 FROM {table_name}")
    
    # b) 버스 정류장 데이터 (유동 인구 활용)
    df_bus = load_data_to_df(conn, 'bus_routes2', 
                             query="SELECT 정류장명, 경도, 위도 FROM {table_name}")
    
    # c) 강원도 사업체 상세 목록 (비교 및 목록용)
    df_kangwon = load_data_to_df(conn, 'kangwondo_businesses',
                                 query="SELECT 업종, 업소명, 주소, 연락처 FROM {table_name}")
    
    # d) 상권 기본 정보 (대시보드 통계용)
    df_area_info = load_data_to_df(conn, 'commercial_area',
                                   query="SELECT 상권업종대분류명, 상권업종중분류명, 시군구명, 경도, 위도 FROM {table_name}")

    conn.close()
    
    # --- 데이터 정리 및 컬럼명 통일 ---
    
    # 1. 사업체 위치 컬럼 통일 및 좌표 정제
    if not df_biz_coords.empty:
        df_biz_coords.columns = ['type', 'name', 'lat', 'lon']
        df_biz_coords = df_biz_coords.dropna(subset=['lat', 'lon'])
        df_biz_coords = df_biz_coords[(df_biz_coords['lat'] > 36) & (df_biz_coords['lon'] > 126)] # 이상치 필터링
        
    # 2. 버스 데이터 컬럼 통일
    if not df_bus.empty:
        df_bus.columns = ['station_name', 'lon', 'lat']
        df_bus = df_bus.dropna(subset=['lon', 'lat'])
        
    # 3. 강원도 사업체 컬럼 통일
    if not df_kangwon.empty:
        df_kangwon.columns = ['type', 'name', 'address', 'contact']
        
    # 4. 상권 정보 컬럼 통일
    if not df_area_info.empty:
        df_area_info.columns = ['main_type', 'sub_type', 'city', 'lon', 'lat']

    return df_biz_coords, df_bus, df_kangwon, df_area_info

# 데이터 로드
df_businesses, df_bus_stations, df_all_businesses, df_area_info = prepare_ui_data()

# --- 2. Streamlit UI/UX 레이아웃 ---

st.set_page_config(layout="wide")
st.title("💰 춘천시 소상공인 창업 및 경쟁 분석 대시보드")
st.markdown("데이터 기반 의사결정을 위한 춘천시 상권 및 경쟁사 정보 제공")

st.sidebar.header("대시보드 설정")

tab1, tab2, tab3 = st.tabs(["🗺️ 지도 기반 상권 분석", "📊 업종별 비교 분석", "📄 상세 데이터 목록"])

# --- 3. 탭별 UI/UX 구현 ---

with tab1: # 🗺️ 지도 기반 상권 분석
    st.header("1. 지도 기반 상권 및 유동 인구 분석")
    
    # 사이드바에서 필터링 설정
    st.sidebar.subheader("지도 필터 설정")
    selected_type = st.sidebar.selectbox(
        '사업체 업종 선택:',
        options=['전체'] + df_businesses['type'].unique().tolist()
    )
    show_bus = st.sidebar.checkbox('🚌 버스 정류장 위치 표시 (유동인구)', value=False)
    
    # 3-1. 필터링 적용
    df_filtered = df_businesses
    if selected_type != '전체':
        df_filtered = df_businesses[df_businesses['type'] == selected_type]

    # 3-2. 지도 생성 (춘천시 중심)
    if not df_businesses.empty:
        center_lat = df_businesses['lat'].mean()
        center_lon = df_businesses['lon'].mean()
        m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

        # 사업체 마커 추가
        for idx, row in df_filtered.iterrows():
            folium.CircleMarker(
                location=[row['lat'], row['lon']],
                radius=4,
                popup=f"**{row['name']}** ({row['type']})",
                color='#FF0000',
                fill=True,
                fill_color='#FF0000'
            ).add_to(m)

        # 버스 정류장 마커 추가
        if show_bus and not df_bus_stations.empty:
            for idx, row in df_bus_stations.iterrows():
                folium.Marker(
                    location=[row['lat'], row['lon']],
                    popup=f"정류장: {row['station_name']}",
                    icon=folium.Icon(color='blue', icon='bus', prefix='fa')
                ).add_to(m)

        st.caption(f"지도에 표시된 사업체 수: **{len(df_filtered)}**개")
        folium_static(m, width=900, height=500)
    else:
        st.error("지도 시각화를 위한 사업체 데이터가 충분하지 않습니다.")

with tab2: # 📊 업종별 비교 분석
    st.header("2. 지역별 시장 포화도 및 주요 업종 비교")
    
    if not df_all_businesses.empty:
        # 3-3. 데이터 전처리: 지역 컬럼 추가 (춘천/강원도 구분)
        df_all_businesses['region'] = df_all_businesses['address'].apply(
            lambda x: '춘천시' if '춘천시' in x else '강원도 타 지역'
        )
        
        # 업종별 개수 계산
        biz_counts = df_all_businesses.groupby(['region', 'type']).size().reset_index(name='count')
        
        # 상위 10개 업종만 분석
        top_types = biz_counts.groupby('type')['count'].sum().nlargest(10).index
        biz_counts_top = biz_counts[biz_counts['type'].isin(top_types)]
        
        st.subheader("강원도/춘천시 상위 10개 업종 분포 비교")
        
        # Bar Chart (막대 그래프)
        st.bar_chart(
            data=biz_counts_top,
            x='type',
            y='count',
            color='region',
            use_container_width=True
        )
        st.caption("특정 업종의 춘천 내 밀집도(경쟁 포화도)를 강원도 전체와 비교하여 확인하세요.")
    else:
        st.error("비교 분석을 위한 강원도 사업체 데이터가 충분하지 않습니다.")

with tab3: # 📄 상세 데이터 목록
    st.header("3. 강원도 사업체 상세 목록 검색 및 다운로드")
    
    if not df_all_businesses.empty:
        # 3-4. 검색 및 필터링
        search_term = st.text_input('업소명, 주소 또는 업종 검색:')
        
        df_search = df_all_businesses
        if search_term:
            df_search = df_all_businesses[
                df_all_businesses['name'].str.contains(search_term, case=False, na=False) |
                df_all_businesses['address'].str.contains(search_term, case=False, na=False) |
                df_all_businesses['type'].str.contains(search_term, case=False, na=False)
            ]
        
        # 3-5. 상세 데이터 테이블 표시
        st.dataframe(df_search[['type', 'name', 'address', 'contact']], use_container_width=True)
        st.caption(f"표시된 사업체 수: {len(df_search)}개")
        
        # 데이터 다운로드 기능
        csv = df_search.to_csv(index=False).encode('utf-8')
        st.download_button(
            label="⬇️ 현재 목록 CSV 다운로드",
            data=csv,
            file_name='filtered_businesses_list.csv',
            mime='text/csv',
        )
    else:
        st.error("상세 목록을 표시할 데이터가 없습니다.")

ModuleNotFoundError: No module named 'streamlit'

In [5]:
!pip3 install streamlit-folium


  Using cached streamlit-1.51.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached markupsafe-3.0.3-cp314-cp314-macosx_11_0_arm64.whl.metadata (2.7 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-6.2.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached pillow-12.0.0-cp314-cp314-macosx_11_0_arm64.whl.metadata (8.8 kB)
  Using cached protobuf-6.33.1-cp39-abi3-macosx_10_9_universal2.whl.metadata (593 bytes)
  Using cached pyarrow-21.0.0.tar.gz (1.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.m